# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [104]:

import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


# URL de la página que queremos scrapear
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

# Realizar la solicitud GET
html_data = requests.get(url, timeout=10).text

# Si la solicitud es denegada, usar headers para evitar el bloqueo
if "403 Forbidden" in html_data:
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"
    }
    response = requests.get(url, headers=headers, timeout=15)
    response.raise_for_status()  # Lanza una excepción si la solicitud no fue exitosa
    html_data = response.text

print("PASO 1")
print(" ")
html_data


PASO 1
 


'<html>\r\n<head>\r\n\n</head>\r\n<body>\r\n<div style="margin: 50px auto; width: 50%; border: 1px solid #dfdfdf; padding: 20px 50px 30px 50px; font-family:helvetica;">\r\n\r\n<h1>We do not allow automated access to our servers.</h1>\r\n<h2><p>Automated access to our data is prohibited by our data provider.</p>  \r\n<p>If you are a user attempting to access the site via a browser, please follow this process to regain access:</p>\r\n\r\n<ul>\r\n  <li>Go to <a href=\'https://whatismyipaddress.com/\' target=\'_blank\' rel=\'noopener noreferrer\'>whatismyipaddress</a> and obtain your IPv4 address</li>\r\n  <li>Email us your IPv4 address at <a href="/cdn-cgi/l/email-protection" class="__cf_email__" data-cfemail="4821262e270825292b3a273c3a2d262c3b66262d3c">[email&#160;protected]</a></li>\r\n  <li>We will add you to our whitelist within 24 hours</li>\r\n</ul>\r\n</h2>\r\n</div>\r\n<script data-cfasync="false">!function(){"use strict";function e(e){try{if("undefined"==typeof console)return;"er

In [105]:
# Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(html_data, "html.parser")
soup


<html>
<head>
</head>
<body>
<div style="margin: 50px auto; width: 50%; border: 1px solid #dfdfdf; padding: 20px 50px 30px 50px; font-family:helvetica;">
<h1>We do not allow automated access to our servers.</h1>
<h2><p>Automated access to our data is prohibited by our data provider.</p>
<p>If you are a user attempting to access the site via a browser, please follow this process to regain access:</p>
<ul>
<li>Go to <a href="https://whatismyipaddress.com/" rel="noopener noreferrer" target="_blank">whatismyipaddress</a> and obtain your IPv4 address</li>
<li>Email us your IPv4 address at <a class="__cf_email__" data-cfemail="4821262e270825292b3a273c3a2d262c3b66262d3c" href="/cdn-cgi/l/email-protection">[email protected]</a></li>
<li>We will add you to our whitelist within 24 hours</li>
</ul>
</h2>
</div>
<script data-cfasync="false">!function(){"use strict";function e(e){try{if("undefined"==typeof console)return;"error"in console?console.error(e):console.log(e)}catch(e){}}function t(e,t){v

In [106]:
# Buscar todas las tablas en la página
tables = soup.find_all("table")
tables


[]

In [107]:
# Buscar el encabezado de la tabla "Revenue (Quarterly) Chart"
header = soup.find("h3", class_="panel-content", string="Revenue (Quarterly) Chart")
if header:
    # Encontrar la tabla que sigue inmediatamente después del encabezado
    table = header.find_next("table")
else:
    raise ValueError("No se encontró el encabezado de 'Revenue (Quarterly) Chart'.")


ValueError: No se encontró el encabezado de 'Revenue (Quarterly) Chart'.

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

# URL de la página que queremos scrapear
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

# Realizar la solicitud GET
html_data = requests.get(url, timeout=10).text

# Si la solicitud es denegada, usar headers para evitar el bloqueo
if "403 Forbidden" in html_data:
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"
    }
    response = requests.get(url, headers=headers, timeout=15)
    response.raise_for_status()
    html_data = response.text

# Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(html_data, "html.parser")

# Buscar el encabezado de la tabla de ingresos trimestrales
header = soup.find("h3", class_="panel-title", string="Historical Revenue (Quarterly) Data")
if header:
    # Encontrar la tabla que sigue inmediatamente después del encabezado
    table = header.find_next("table")
else:
    raise ValueError("No se encontró el encabezado de 'Historical Revenue (Quarterly) Data'.")

# Paso 4: Procesar el marco de datos
# Construir el DataFrame sin pd.concat en un bucle
data = []

for row in table.tbody.find_all("tr"):
    col = row.find_all("td")
    if col:
        Date = col[0].text.strip()
        Revenue = col[1].text.strip().replace("$", "").replace(",", "")
        if Revenue:  # Solo añadir si Revenue no está vacío
            data.append({"Date": Date, "Revenue": Revenue})

# Convertir lista de diccionarios a DataFrame de una sola vez
tesla_revenue = pd.DataFrame(data)
tesla_revenue["Date"] = pd.to_datetime(tesla_revenue["Date"])
tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"], errors='coerce').fillna(0).astype(int)

# Paso 5: Almacenar los datos en SQLite
connection = sqlite3.connect("Tesla.db")
cursor = connection.cursor()

# Crear la tabla 'revenue' si no existe
cursor.execute("DROP TABLE IF EXISTS revenue")  # Para evitar duplicados en pruebas
cursor.execute("CREATE TABLE revenue (Date TEXT, Revenue INTEGER)")

# Insertar datos en SQLite con un solo comando executemany
tesla_tuples = list(tesla_revenue.itertuples(index=False, name=None))
cursor.executemany("INSERT INTO revenue VALUES (?, ?)", tesla_tuples)

# Confirmar los cambios en la base de datos y cerrar la conexión
connection.commit()
connection.close()

# Paso 6: Visualización de datos
# Visualización de series de tiempos de ingresos trimestrales
fig, axis = plt.subplots(figsize=(10, 5))
sns.lineplot(data=tesla_revenue, x="Date", y="Revenue", ax=axis)
axis.set_title("Ingresos trimestrales de Tesla")
plt.tight_layout()
plt.show()

# Beneficio bruto anual
fig, axis = plt.subplots(figsize=(10, 5))

# Agrupar los ingresos anualmente
tesla_revenue_yearly = tesla_revenue.groupby(tesla_revenue["Date"].dt.year).sum().reset_index()

# Filtrar hasta 2022 y graficar
sns.barplot(data=tesla_revenue_yearly[tesla_revenue_yearly["Date"] < 2023], x="Date", y="Revenue", ax=axis)
axis.set_title("Beneficio bruto anual de Tesla")
plt.tight_layout()
plt.show()

# Beneficio bruto mensual
fig, axis = plt.subplots(figsize=(10, 5))

# Agrupar los ingresos mensualmente (agregación en todos los años)
tesla_revenue_monthly = tesla_revenue.groupby(tesla_revenue["Date"].dt.month).sum().reset_index()

# Graficar ingresos por mes
sns.barplot(data=tesla_revenue_monthly, x="Date", y="Revenue", ax=axis)
axis.set_title("Beneficio bruto mensual promedio de Tesla")
plt.tight_layout()
plt.show()